In [2]:
import re
import os
import spacy
import pandas as pd
from spacy.lang.de import German
import xml.etree.ElementTree as ET
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass

nlp = German()
nlp.add_pipe("sentencizer", config={"punct_chars": [".", "?", "!"]})


# ============================================================================
# SHARED UTILITIES
# ============================================================================

@dataclass
class SentencePair:
    """Represents a source-target sentence pair with metadata."""
    src: str
    tgt: str
    has_correction: bool
    has_foreign: bool
    
    def to_tuple(self):
        return (self.src, self.tgt, self.has_correction, self.has_foreign)


class TextBuilder:
    """
    Handles text accumulation with proper spacing preservation.
    CRITICAL: Respects XML whitespace at all times.
    """
    def __init__(self):
        self.parts = []
    
    def add_text(self, text: str, merge: bool = False):
        """
        Add text with intelligent spacing.
        
        Args:
            text: Text to add
            merge: If True, merge directly without space (for mid-word situations)
        """
        if not text:
            return
        
        # Filter out "unreadable" literals
        text = re.sub(r'\bunreadable\b', '', text, flags=re.IGNORECASE)
        text = re.sub(r'unreadable', '', text, flags=re.IGNORECASE)
        text = text.strip()
        
        if not text:
            return
        
        if not self.parts:
            self.parts.append(text)
            return
        
        if merge:
            # Direct merge for mid-word cases
            self.parts.append(text)
        else:
            # Add space if last part doesn't end with one
            if self.parts[-1] and not self.parts[-1].endswith(' '):
                self.parts.append(' ')
            self.parts.append(text)
    
    def add_space(self):
        """Explicitly add a space."""
        if self.parts and not self.parts[-1].endswith(' '):
            self.parts.append(' ')
    
    def add_marker(self, marker: str):
        """Add a marker (like <SENTBREAK> or <FOREIGN>)."""
        self.parts.append(marker)
    
    def get_text(self) -> str:
        """Get accumulated text with cleanup."""
        text = ''.join(self.parts)
        # Clean up multiple spaces but preserve single spaces
        text = re.sub(r' +', ' ', text)
        # Remove spaces before punctuation
        text = re.sub(r'\s+([.:;!?,])', r'\1', text)
        return text.strip()


def has_leading_whitespace(text: Optional[str]) -> bool:
    """Check if text starts with whitespace in original XML."""
    return text is not None and len(text) > 0 and text[0].isspace()


def has_trailing_whitespace(text: Optional[str]) -> bool:
    """Check if text ends with whitespace in original XML."""
    return text is not None and len(text) > 0 and text[-1].isspace()


def strip_namespace(tag: str) -> str:
    """Remove XML namespace from tag."""
    if '}' in tag:
        return tag.split('}', 1)[1]
    return tag


def has_sentence_ending(text: str) -> bool:
    """Check if text ends with sentence-ending punctuation."""
    if not text:
        return False
    return bool(re.search(r'[.!?]\s*$', text.strip()))


def spacy_sent(text: str) -> List[str]:
    """Split German text into sentences using spaCy."""
    if not text or not text.strip():
        return []

    # Pre-split on double/triple periods
    text = re.sub(r'\.{2,}', '.<SPLIT>', text)
    # Add space after sentence-ending punctuation if missing
    text = re.sub(r'([.!?]+)([A-ZÄÖÜ])', r'\1 \2', text)

    chunks = text.split('<SPLIT>')
    all_sentences = []
    
    for chunk in chunks:
        if not chunk.strip():
            continue
        
        clean = re.sub(r"<[^>]+>", " ", chunk)
        clean = re.sub(r"[ ]+", " ", clean)
        clean = re.sub(r"\n{2,}", "\n<PAR>\n", clean)
        clean = clean.strip()
        
        doc = nlp(clean)
        out = []
        for sent in doc.sents:
            s = sent.text.strip()
            if not s or re.fullmatch(r"[\.\?!]+", s) or re.fullmatch(r"\d+[\.\)]\s*$", s):
                continue
            s = s.replace("<PAR>", "").strip()
            if s:
                out.append(s)
        
        all_sentences.extend(out)

    # Merge fragments
    merged = []
    buffer = ""
    for sentence in all_sentences:
        s_strip = sentence.strip()
        if not s_strip:
            continue
        if buffer:
            if s_strip[0].islower() or not re.search(r'[.!?]$', buffer):
                buffer += " " + s_strip
            else:
                merged.append(buffer)
                buffer = s_strip
        else:
            buffer = s_strip
    
    if buffer:
        merged.append(buffer)

    return merged


# ============================================================================
# KOLIPSI EXTRACTION
# ============================================================================

def extract_kolipsi(element) -> Tuple[str, str, bool]:
    """
    Extract src and tgt from Kolipsi element.
    Returns (src_text, tgt_text, has_corrections)
    """
    src_builder = TextBuilder()
    tgt_builder = TextBuilder()
    has_corrections = False

    def get_element_text(elem):
        """Get all text from element and descendants."""
        if elem is None:
            return ""
        return ''.join(elem.itertext()).strip()

    def get_original_form_text(elem):
        """Extract text from originalForm, handling nested structures."""
        if elem is None:
            return ""

        parts = []

        def recurse_original(node):
            tag = strip_namespace(node.tag).lower()

            if node.text and node.text.strip():
                parts.append(node.text.strip())

            if tag == "overwrite":
                over = None
                for child in node:
                    if strip_namespace(child.tag).lower() == "over":
                        over = child
                        break
                if over is not None and over.text:
                    parts.append(over.text.strip())
                if node.tail and node.tail.strip():
                    parts.append(node.tail.strip())
                return

            if tag == "palimpsest":
                palimpsest_text = ''.join(node.itertext()).strip()
                if palimpsest_text:
                    parts.append(palimpsest_text)
                if node.tail and node.tail.strip():
                    if has_leading_whitespace(node.tail):
                        parts.append(' ')
                    parts.append(node.tail.strip())
                return

            for child in node:
                recurse_original(child)

            if node.tail and node.tail.strip():
                parts.append(node.tail.strip())

        recurse_original(elem)
        return ''.join(parts)

    def recurse(node, src: TextBuilder, tgt: TextBuilder):
        nonlocal has_corrections
        tag = strip_namespace(node.tag).lower()

        # ERROR / OVER_CAPITALISATION / E
        if tag in ("error", "over_capitalisation", "e"):
            has_corrections = True
            original = None
            target = None

            for child in node:
                child_tag = strip_namespace(child.tag).lower()
                if child_tag == "originalform":
                    original = child
                elif child_tag == "targetform":
                    target = child

            orig_text = get_original_form_text(original) if original is not None else ""
            tgt_text = get_element_text(target) if target is not None else ""

            # Check for sentence break
            prev_src = src.get_text()
            if (orig_text and tgt_text
                and len(orig_text) > 0 and len(tgt_text) > 0
                and orig_text[0].islower() != tgt_text[0].islower()
                and has_sentence_ending(prev_src)):
                src.add_marker(" <SENTBREAK> ")
                tgt.add_marker(" <SENTBREAK> ")

            if orig_text:
                src.add_text(orig_text)
            if tgt_text:
                tgt.add_text(tgt_text)

            # Handle tail with proper spacing
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                tail_text = node.tail.strip()
                if tail_text:
                    src.add_text(tail_text, merge=True)
                    tgt.add_text(tail_text, merge=True)
            return

        # PALIMPSEST
        elif tag == "palimpsest":
            has_errors = any(
                strip_namespace(child.tag).lower() in ("error", "over_capitalisation", "e")
                for child in node
            )

            has_strikeover = any(
                strip_namespace(child.tag).lower() == "strikeover"
                for child in node
            )

            # Strikeover case
            if has_strikeover:
                if node.text and node.text.strip():
                    src.add_text(node.text.strip())
                    tgt.add_text(node.text.strip())

                for child in node:
                    child_tag = strip_namespace(child.tag).lower()
                    if child_tag == "strikeover":
                        expansion_parts = []
                        for grandchild in child:
                            if strip_namespace(grandchild.tag).lower() == "expansion" and grandchild.text:
                                expansion_parts.append(grandchild.text)

                        if expansion_parts:
                            merged = ''.join(expansion_parts)
                            src.add_text(merged, merge=True)
                            tgt.add_text(merged, merge=True)

                        if child.tail:
                            if has_leading_whitespace(child.tail):
                                src.add_space()
                                tgt.add_space()
                            if child.tail.strip():
                                src.add_text(child.tail.strip(), merge=True)
                                tgt.add_text(child.tail.strip(), merge=True)
                    else:
                        recurse(child, src, tgt)

                if node.tail:
                    if has_leading_whitespace(node.tail):
                        src.add_space()
                        tgt.add_space()
                    if node.tail.strip():
                        src.add_text(node.tail.strip(), merge=True)
                        tgt.add_text(node.tail.strip(), merge=True)
                return

            # No errors case - check XML spacing
            if not has_errors:
                if node.text and node.text.strip():
                    # Check if this is mid-word by looking at surrounding whitespace
                    merge_before = src.parts and src.parts[-1] and not src.parts[-1].endswith(' ')
                    src.add_text(node.text.strip(), merge=merge_before)
                    tgt.add_text(node.text.strip(), merge=merge_before)

                for child in node:
                    recurse(child, src, tgt)

                if node.tail:
                    if has_leading_whitespace(node.tail):
                        src.add_space()
                        tgt.add_space()
                    if node.tail.strip():
                        # Check if tail should merge (no leading space in XML)
                        merge_tail = not has_leading_whitespace(node.tail)
                        src.add_text(node.tail.strip(), merge=merge_tail)
                        tgt.add_text(node.tail.strip(), merge=merge_tail)
                return

            # Has errors case
            if node.text and node.text.strip():
                src.add_text(node.text.strip())
                tgt.add_text(node.text.strip())

            for child in node:
                child_tag = strip_namespace(child.tag).lower()

                if child_tag in ("error", "over_capitalisation", "e"):
                    has_corrections = True
                    original = None
                    target = None

                    for grandchild in child:
                        grandchild_tag = strip_namespace(grandchild.tag).lower()
                        if grandchild_tag == "originalform":
                            original = grandchild
                        elif grandchild_tag == "targetform":
                            target = grandchild

                    orig_text = get_original_form_text(original)
                    if orig_text:
                        src.add_text(orig_text)

                    tgt_text = get_element_text(target)
                    if tgt_text:
                        tgt.add_text(tgt_text)

                    if child.tail:
                        if has_leading_whitespace(child.tail):
                            src.add_space()
                            tgt.add_space()
                        if child.tail.strip():
                            merge_tail = not has_leading_whitespace(child.tail)
                            src.add_text(child.tail.strip(), merge=merge_tail)
                            tgt.add_text(child.tail.strip(), merge=merge_tail)
                else:
                    recurse(child, src, tgt)

            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # CORRECTION
        elif tag == "correction":
            deletion_text = ""
            insertion_text = ""
            
            for child in node:
                child_tag = strip_namespace(child.tag).lower()
                
                if child_tag == "deletion":
                    if child.text and child.text.strip():
                        deletion_text = child.text.strip()
                
                elif child_tag == "insertion":
                    if child.text and child.text.strip():
                        insertion_text = child.text.strip()
                    for grandchild in child:
                        recurse(grandchild, src, tgt)
            
            if deletion_text and insertion_text:
                src.add_text(insertion_text, merge=True)
                tgt.add_text(insertion_text, merge=True)
            elif insertion_text and not deletion_text:
                needs_space_before = False
                if src.parts:
                    last_part = src.parts[-1]
                    if last_part and last_part[-1].islower():
                        words = last_part.split()
                        if words and len(words[-1]) > 2:
                            needs_space_before = True
                
                if needs_space_before:
                    src.add_space()
                    tgt.add_space()
                
                src.add_text(insertion_text, merge=True)
                tgt.add_text(insertion_text, merge=True)
            
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    src.add_text(node.tail.strip(), merge=True)
                    tgt.add_text(node.tail.strip(), merge=True)
            return

        # REDUCTION
        elif tag == "reduction":
            unfolded = None
            for child in node:
                child_tag = strip_namespace(child.tag).lower()
                if child_tag == "unfoldedform":
                    unfolded = child
                    break
            
            if unfolded is not None and unfolded.text:
                unfolded_text = unfolded.text.strip()
                needs_space = False
                if src.parts:
                    last_part = src.parts[-1]
                    if last_part and last_part[-1].isalpha():
                        words = last_part.split()
                        if words and len(words[-1]) > 2:
                            needs_space = True
                
                if needs_space:
                    src.add_space()
                    tgt.add_space()
                
                src.add_text(unfolded_text)
                tgt.add_text(unfolded_text)
            
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # AMBIGUOUS
        elif tag == "ambiguous":
            first_alternative = None
            for child in node:
                child_tag = strip_namespace(child.tag).lower()
                if child_tag == "alternative":
                    first_alternative = child
                    break
            
            if first_alternative is not None and first_alternative.text:
                alt_text = first_alternative.text.strip()
                needs_space = False
                if src.parts:
                    last_part = src.parts[-1]
                    if last_part and last_part[-1].isalpha():
                        words = last_part.split()
                        if words and len(words[-1]) > 2:
                            needs_space = True
                
                if needs_space:
                    src.add_space()
                    tgt.add_space()
                
                src.add_text(alt_text)
                tgt.add_text(alt_text)
            
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # STRIKEOVER
        elif tag == "strikeover":
            expansions = [child.text for child in node
                          if strip_namespace(child.tag).lower() == "expansion" and child.text]
        
            merged = "".join(expansions)
        
            should_merge = (
                src.parts
                and src.parts[-1]
                and not src.parts[-1].endswith((" ", "\n"))
            )
        
            if should_merge:
                src.add_text(merged, merge=True)
                tgt.add_text(merged, merge=True)
            else:
                src.add_text(merged)
                tgt.add_text(merged)
        
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # OVERWRITE
        elif tag == "overwrite":
            over = None
            for child in node:
                child_tag = strip_namespace(child.tag).lower()
                if child_tag == "over":
                    over = child
                    break

            over_text = over.text if over is not None and over.text else ""

            if over_text:
                src.add_text(over_text, merge=True)
                tgt.add_text(over_text, merge=True)

            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    src.add_text(node.tail.strip(), merge=True)
                    tgt.add_text(node.tail.strip(), merge=True)
            return

        # FOREIGN_WORD
        elif tag == "foreign_word":
            foreign_text = node.text.strip() if node.text and node.text.strip() else ""
            
            if foreign_text:
                marked_word = f'FOREIGNWORDSTART{foreign_text}FOREIGNWORDEND'
                src.add_text(marked_word)
                tgt.add_text(marked_word)
            
            for child in node:
                recurse(child, src, tgt)
            
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # IGNORE
        elif tag in ("symbol", "emoticon", "unreadable"):
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # PAR
        elif tag == "par":
            src.add_marker(" <SENTBREAK> ")
            tgt.add_marker(" <SENTBREAK> ")
            
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    src.add_text(node.tail.strip())
                    tgt.add_text(node.tail.strip())
            return

        # SPACEWRAPPER
        elif tag == "spacewrapper":
            src.add_space()
            tgt.add_space()
            
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)
            return

        # GREETING / CLOSING / ENTITY
        elif tag in ("greeting", "closing", "entity"):
            if node.text and node.text.strip():
                src.add_text(node.text.strip())
                tgt.add_text(node.text.strip())
        
            for child in node:
                recurse(child, src, tgt)
        
            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    src.add_text(node.tail.strip())
                    tgt.add_text(node.tail.strip())
            else:
                # No tail means next sibling comes directly after
                src.add_space()
                tgt.add_space()
            return

        # OTHER (default handler)
        else:
            if node.text:
                text_stripped = node.text.strip()
                has_trailing = has_trailing_whitespace(node.text)
                
                if text_stripped:
                    src.add_text(text_stripped)
                    tgt.add_text(text_stripped)
                
                if has_trailing:
                    src.add_space()
                    tgt.add_space()

            for child in node:
                recurse(child, src, tgt)

            if node.tail:
                if has_leading_whitespace(node.tail):
                    src.add_space()
                    tgt.add_space()
                if node.tail.strip():
                    merge_tail = not has_leading_whitespace(node.tail)
                    src.add_text(node.tail.strip(), merge=merge_tail)
                    tgt.add_text(node.tail.strip(), merge=merge_tail)

    recurse(element, src_builder, tgt_builder)
    return src_builder.get_text(), tgt_builder.get_text(), has_corrections


def extract_kolipsi_sentences(element) -> List[SentencePair]:
    """Extract sentence pairs from Kolipsi element."""
    src_full, tgt_full, _ = extract_kolipsi(element)

    if not src_full and not tgt_full:
        return []

    src_chunks = [s.strip() for s in src_full.split('<SENTBREAK>') if s.strip()]
    tgt_chunks = [s.strip() for s in tgt_full.split('<SENTBREAK>') if s.strip()]

    if len(src_chunks) != len(tgt_chunks):
        max_chunks = max(len(src_chunks), len(tgt_chunks))
        src_chunks.extend([''] * (max_chunks - len(src_chunks)))
        tgt_chunks.extend([''] * (max_chunks - len(tgt_chunks)))

    pairs = []
    for src_chunk, tgt_chunk in zip(src_chunks, tgt_chunks):
        if not src_chunk and not tgt_chunk:
            continue

        # Detect foreign words at chunk level but clean before splitting
        has_foreign_in_chunk = ('FOREIGNWORDSTART' in src_chunk or 
                               'FOREIGNWORDSTART' in tgt_chunk)
        
        src_chunk = re.sub(r'FOREIGNWORDSTART(.*?)FOREIGNWORDEND', r'\1', src_chunk)
        tgt_chunk = re.sub(r'FOREIGNWORDSTART(.*?)FOREIGNWORDEND', r'\1', tgt_chunk)
        
        src_chunk = re.sub(r'\s+', ' ', src_chunk).strip()
        tgt_chunk = re.sub(r'\s+', ' ', tgt_chunk).strip()

        src_sents = spacy_sent(src_chunk) if src_chunk else []
        tgt_sents = spacy_sent(tgt_chunk) if tgt_chunk else []

        if not src_sents and not tgt_sents:
            continue

        max_len = max(len(src_sents), len(tgt_sents))
        for i in range(max_len):
            src_sent = src_sents[i] if i < len(src_sents) else ""
            tgt_sent = tgt_sents[i] if i < len(tgt_sents) else ""
            
            has_correction = (src_sent.strip() != tgt_sent.strip())
            
            if src_sent or tgt_sent:
                pairs.append(SentencePair(
                    src=src_sent,
                    tgt=tgt_sent,
                    has_correction=has_correction,
                    has_foreign=has_foreign_in_chunk
                ))

    return pairs


# ============================================================================
# LEONIDE EXTRACTION
# ============================================================================

def extract_leonide(paragraph) -> Tuple[str, str, bool]:
    """Extract text from LEONIDE paragraph."""
    src_builder = TextBuilder()
    tgt_builder = TextBuilder()
    has_corrections = False

    def process_node(node, src: TextBuilder, tgt: TextBuilder):
        nonlocal has_corrections
        
        if node.text and node.text.strip():
            src.add_text(node.text.strip())
            tgt.add_text(node.text.strip())

        for child in node:
            tag = child.tag.lower()

            # FOREIGN WORD
            if 'tran_foreign_word' in tag:
                foreign_text = child.text.strip() if child.text and child.text.strip() else ""
                
                if foreign_text:
                    marked_word = f'FOREIGNWORDSTART{foreign_text}FOREIGNWORDEND'
                    src.add_text(marked_word)
                    tgt.add_text(marked_word)
                
                for grandchild in child:
                    process_node(grandchild, src, tgt)
                
                if child.tail:
                    if has_leading_whitespace(child.tail):
                        src.add_space()
                        tgt.add_space()
                    if child.tail.strip():
                        src.add_text(child.tail.strip())
                        tgt.add_text(child.tail.strip())
                continue

            # SYMBOL / EMOTICON
            if 'tran_symbol' in tag or 'tran_emoticon' in tag:
                if child.tail:
                    if has_leading_whitespace(child.tail):
                        src.add_space()
                        tgt.add_space()
                    if child.tail.strip():
                        src.add_text(child.tail.strip())
                        tgt.add_text(child.tail.strip())
                continue

            # DIV
            if tag == 'div':
                src.add_space()
                tgt.add_space()

                process_node(child, src, tgt)

                if child.tail and child.tail.strip():
                    src.add_space()
                    tgt.add_space()
                    src.add_text(child.tail.strip())
                    tgt.add_text(child.tail.strip())
                else:
                    src.add_space()
                    tgt.add_space()
                continue

            # WORD CORRECTION / AMBIGUOUS
            if 'tran_word_correction' in tag or 'tran_ambiguous' in tag:
                src.add_space()
                tgt.add_space()
                
                if child.text and child.text.strip():
                    src.add_text(child.text.strip())
                    tgt.add_text(child.text.strip())
                    
                for grandchild in child:
                    process_node(grandchild, src, tgt)
                    
                if child.tail:
                    if has_leading_whitespace(child.tail):
                        src.add_space()
                        tgt.add_space()
                    if child.tail.strip():
                        src.add_text(child.tail.strip())
                        tgt.add_text(child.tail.strip())
                continue

            # WORD DELETION
            if 'tran_word_deletion' in tag:
                if child.tail:
                    if has_leading_whitespace(child.tail):
                        src.add_space()
                        tgt.add_space()
                    if child.tail.strip():
                        src.add_text(child.tail.strip())
                        tgt.add_text(child.tail.strip())
                continue

            # ORTH ERROR
            if 'orth_error' in tag:
                has_corrections = True
                src.add_space()
                tgt.add_space()
                
                target_attr = child.get('orth_error_target')
                
                # Check duplicates
                should_add_target = True
                if target_attr and tgt.parts:
                    recent_text = ' '.join(tgt.parts[-3:]) if len(tgt.parts) >= 3 else ' '.join(tgt.parts)
                    if target_attr in recent_text:
                        should_add_target = False
                
                if target_attr and should_add_target:
                    tgt.add_text(target_attr)
                
                # Get all text from inside orth_error
                original_text = ''.join(child.itertext()).strip()
                if original_text:
                    src.add_text(original_text)
                
                # CRITICAL: Check for leading space in tail
                if child.tail:
                    if has_leading_whitespace(child.tail):
                        src.add_space()
                        tgt.add_space()
                    if child.tail.strip():
                        src.add_text(child.tail.strip())
                        tgt.add_text(child.tail.strip())
                continue
                
            # CAPITALISATION
            if 'tran_capitalisation' in tag:
                original_attr = child.text
                target_attr = child.get('tran_capitalisation_target')
                if original_attr:
                    src.add_text(original_attr)
                if target_attr:
                    tgt.add_text(target_attr)
                    
                if child.tail:
                    if has_leading_whitespace(child.tail):
                        src.add_space()
                        tgt.add_space()
                    if child.tail.strip():
                        src.add_text(child.tail.strip())
                        tgt.add_text(child.tail.strip())
                continue

            # Recurse for other tags
            process_node(child, src, tgt)
            
            # Handle tail
            if child.tail:
                if has_leading_whitespace(child.tail):
                    src.add_space()
                    tgt.add_space()
                if child.tail.strip():
                    src.add_text(child.tail.strip())
                    tgt.add_text(child.tail.strip())

    process_node(paragraph, src_builder, tgt_builder)
    return src_builder.get_text(), tgt_builder.get_text(), has_corrections


def extract_leonide_sentences(paragraph) -> List[SentencePair]:
    """Extract sentence pairs from LEONIDE paragraph."""
    src, tgt, _ = extract_leonide(paragraph)

    if not src and not tgt:
        return []

    # Detect foreign words before cleaning markers
    has_foreign = 'FOREIGNWORDSTART' in src or 'FOREIGNWORDSTART' in tgt
    
    # Clean markers
    src = re.sub(r'FOREIGNWORDSTART(.*?)FOREIGNWORDEND', r'\1', src)
    tgt = re.sub(r'FOREIGNWORDSTART(.*?)FOREIGNWORDEND', r'\1', tgt)
    
    src_sents = spacy_sent(src)
    tgt_sents = spacy_sent(tgt)

    max_len = max(len(src_sents), len(tgt_sents))
    pairs = []
    
    for i in range(max_len):
        src_sent = src_sents[i] if i < len(src_sents) else ""
        tgt_sent = tgt_sents[i] if i < len(tgt_sents) else ""
        
        has_correction = (src_sent.strip() != tgt_sent.strip())
        
        if src_sent or tgt_sent:
            pairs.append(SentencePair(
                src=src_sent,
                tgt=tgt_sent,
                has_correction=has_correction,
                has_foreign=has_foreign
            ))

    return pairs


# ============================================================================
# MAIN EXTRACTION PIPELINE
# ============================================================================

def inject_spaces_between_tags(xml_string: str) -> str:
    """Inject explicit SPACEWRAPPER nodes for meaningful spaces in XML."""
    injected_count = 0
    
    def replacer_text_space_tag(match):
        nonlocal injected_count
        before_gt = match.group(1)
        text = match.group(2)
        spaces = match.group(3)
        after_lt = match.group(4)
        
        if '\n' in spaces or not text.strip():
            return match.group(0)
        
        injected_count += 1
        return f'{before_gt}{text}<SPACEWRAPPER> </SPACEWRAPPER>{after_lt}'
    
    result = re.sub(r'(>)([^<\n]*?\S[^<\n]*?)([ \t]+)(<)', replacer_text_space_tag, xml_string)
    
    def replacer_tag_space_tag(match):
        nonlocal injected_count
        before = match.group(1)
        space = match.group(2)
        after = match.group(3)
        
        if '\n' in space:
            return match.group(0)
        
        injected_count += 1
        return f'{before}<SPACEWRAPPER> </SPACEWRAPPER>{after}'
    
    result = re.sub(r'(>)([ \t]+)(<)', replacer_tag_space_tag, result)
    
    return result


def extract_from_xml(xml_content: str, corpus_type: str) -> List[SentencePair]:
    """Main extraction function."""
    # Inject space wrappers
    xml_content = inject_spaces_between_tags(xml_content)

    try:
        root = ET.fromstring(xml_content)
    except ET.ParseError as e:
        print(f"[ERROR] XML Parse Error: {e}")
        return []

    if corpus_type == "LEONIDE":
        paras = root.findall('.//{http://www.eurac.edu/transcanno}paragraph') or root.findall('.//paragraph')
        all_pairs = []
        for para in paras:
            pairs = extract_leonide_sentences(para)
            all_pairs.extend(pairs)
        return all_pairs

    else:  # Kolipsi
        if "Kolipsi_1" in corpus_type or "Kolipsi-1" in corpus_type:
            ns_body = '{http://www.eurac.edu/kolipsi}body'
        else:
            ns_body = '{http://www.eurac.edu/kolipsi_II}body'
    
        body = root.find(f'.//{ns_body}')
        if body is None:
            body = root.find('.//body')
    
        if body is None:
            print(f"[ERROR] No body element found")
            return []
        
        exercises = body.findall('.//exercise')
        if not exercises:
            exercises = [body]

        all_pairs = []
        for ex in exercises:
            if ex is None:
                continue
            pairs = extract_kolipsi_sentences(ex)
            all_pairs.extend(pairs)

        return all_pairs


def clean_sentence_pairs(pairs: List[SentencePair]) -> List[SentencePair]:
    """Clean and deduplicate sentence pairs."""
    cleaned = []
    seen_pairs = set()
    empty_regex = r"^\s*[\.\?!]*\s*$"

    for pair in pairs:
        # Skip foreign words
        if pair.has_foreign:
            continue
        
        src = re.sub(r"\s*\n\s*", " ", pair.src).strip()
        tgt = re.sub(r"\s*\n\s*", " ", pair.tgt).strip()

        # Skip asterisks (censored content)
        if '*' in src or '*' in tgt:
            continue

        if re.fullmatch(empty_regex, src) or re.fullmatch(empty_regex, tgt):
            continue

        src = re.sub(r"^\d+[\.\)]\s+", "", src).strip()
        tgt = re.sub(r"^\d+[\.\)]\s+", "", tgt).strip()

        if not src or not tgt:
            continue
        
        src_words = re.findall(r'\b\w+\b', src)
        tgt_words = re.findall(r'\b\w+\b', tgt)
        
        if len(src_words) <= 2 or len(tgt_words) <= 2:
            continue
        
        pair_key = (src.lower(), tgt.lower())
        if pair_key in seen_pairs:
            continue
        
        seen_pairs.add(pair_key)
        cleaned.append(SentencePair(src, tgt, pair.has_correction, pair.has_foreign))

    return cleaned


def process_file(xml_path: str, corpus_type: str) -> List[SentencePair]:
    """Process a single XML file."""
    if not os.path.exists(xml_path):
        raise FileNotFoundError(f"{xml_path} not found")

    with open(xml_path, "r", encoding="utf-8", errors="ignore") as f:
        xml_content = f.read()

    pairs = extract_from_xml(xml_content, corpus_type)
    return clean_sentence_pairs(pairs)


def process_corpora(
    corpus_configs: Dict[str, Dict],
    output_dir: str = "output",
    max_files_per_corpus: Optional[int] = None,
    output_format: str = "both"
) -> pd.DataFrame:
    """Process multiple corpora."""
    os.makedirs(output_dir, exist_ok=True)
    
    all_data = []
    file_id = 1
    
    for corpus_name, cfg in corpus_configs.items():
        print(f"\n--- Processing {corpus_name} ---")

        base_dir = cfg["base_dir"]
        lang_prof = cfg.get("lang_prof", "L2")

        if not os.path.isdir(base_dir):
            print(f"  ERROR: Base directory not found: {base_dir}")
            continue

        xml_members = []
        for root_dir, dirs, files in os.walk(base_dir):
            dirs[:] = [d for d in dirs if d != '.ipynb_checkpoints' and not d.startswith('.')]
            files.sort()
            for f in files:
                if f.lower().endswith(".xml") and not f.lower().endswith(".xml.pretty"):
                    xml_members.append(os.path.join(root_dir, f))

        print(f"  Found {len(xml_members)} XML files")

        if max_files_per_corpus:
            xml_members = xml_members[:max_files_per_corpus]

        corpus_pairs = []
        for idx, member in enumerate(xml_members):
            print(f"   [{idx + 1}/{len(xml_members)}] {member}")

            try:
                pairs = process_file(member, corpus_name)
            except Exception as e:
                print(f"     ERROR: {e}")
                continue
            
            xml_filename = os.path.basename(member)
            for sent_num, pair in enumerate(pairs, start=1):
                all_data.append({
                    'corpus': corpus_name,
                    'lang_prof': lang_prof,
                    'xml_file': xml_filename,
                    'file_id': file_id,
                    'sent_num': sent_num,
                    'src': pair.src,
                    'tgt': pair.tgt,
                    'corrected': pair.has_correction
                })
            
            corpus_pairs.extend(pairs)
            file_id += 1

        # Write TXT output
        if output_format in ["txt", "both"]:
            out_path = os.path.join(output_dir, f"{corpus_name}_full.txt")
            with open(out_path, "w", encoding="utf-8") as fh:
                for pair in corpus_pairs:
                    fh.write(pair.src + "\n")
                    fh.write(pair.tgt + "\n\n")
            print(f"  Wrote {len(corpus_pairs)} pairs to {out_path}")

    df = pd.DataFrame(all_data)
    
    # Write CSV output
    if output_format in ["csv", "both"]:
        csv_path = os.path.join(output_dir, "all_corpora.csv")
        df.to_csv(csv_path, index=False, encoding="utf-8")
        print(f"\n=== Wrote {len(df)} rows to {csv_path} ===")
    
    return df


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    corpus_configs = {
        'LEONIDE': {
            'base_dir': 'LEONIDE/pepper-xml-v1.1/data/DE',
            'lang_prof': 'L2'
        },
        'Kolipsi_1_L2': {
            'base_dir': 'Kolipsi_1/xmlmind-v1.1/data/annotations/L2/DE/files_split_by_exercises',
            'lang_prof': 'L2'
        },
        'Kolipsi_1_L1': {
            'base_dir': 'Kolipsi_1/xmlmind-v1.1/data/annotations/L1/DE/files_split_by_exercises',
            'lang_prof': 'L1'
        },
        'Kolipsi_2': {
            'base_dir': 'Kolipsi_2',
            'lang_prof': 'L2'
        }
    }
    
    OUTPUT_FORMAT = "both"
    
    active_corpora = ["LEONIDE", "Kolipsi_1_L1", "Kolipsi_1_L2", "Kolipsi_2"]  # Add corpus names here to process
    configs_to_run = {k: v for k, v in corpus_configs.items() if k in active_corpora}
    
    if configs_to_run:
        df = process_corpora(
            corpus_configs=configs_to_run,
            output_dir='.',
            output_format=OUTPUT_FORMAT
        )
        
        if not df.empty:
            print("\n=== SUMMARY ===")
            print(f"Total rows: {len(df)}")
            print("\nCorpus breakdown:")
            print(df.groupby(['corpus', 'lang_prof']).size())
            print("\nCorrection statistics:")
            print(df['corrected'].value_counts())
    else:
        print("No corpora selected. Add corpus names to active_corpora list.")


--- Processing LEONIDE ---
  Found 833 XML files
   [1/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_pic_3_57X31A10_107/DE_pic_3_57X31A10_107.xml
   [2/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_2_57Y25A05_62/DE_op_2_57Y25A05_62.xml
   [3/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_pic_2_57Y25A13_59/DE_pic_2_57Y25A13_59.xml
   [4/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_1_57X31A18_100/DE_op_1_57X31A18_100.xml
   [5/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_3_57Y27A08_118/DE_op_3_57Y27A08_118.xml
   [6/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_pic_3_57X33A15_106/DE_pic_3_57X33A15_106.xml
   [7/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_1_57X33A11_8/DE_op_1_57X33A11_8.xml
   [8/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_2_57Y25A10_62/DE_op_2_57Y25A10_62.xml
   [9/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_3_57X32A08_108/DE_op_3_57X32A08_108.xml
   [10/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_3_57Y27A05_118/DE_op_3_57Y27A05_118.xml
   [11/833] LEONIDE/pepper-xml-v1.1/data/DE/DE_op_2_67Y27A19_64

In [4]:
import spacy
import pandas as pd
from IPython.display import display

# Load spaCy with sentencizer
def load_spacy(model="de_core_news_sm"):
    try:
        nlp = spacy.load(model, disable=["tagger", "parser", "ner", "lemmatizer"])
    except:
        nlp = spacy.blank("de")
    if "sentencizer" not in nlp.pipe_names:
        nlp.add_pipe("sentencizer")
    return nlp

nlp = load_spacy()
nlp.max_length = 2_000_000

# Process one corpus file (TXT)
def process_corpus_spacy(path: str):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read().strip()
    if not text:
        return {"n_sentences": 0, "words": 0, "unique_tokens": 0, "avg_words_per_sentence": 0}
    
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    tokens = [tok.text for tok in doc if tok.is_alpha]
    num_sent = len(sentences)
    num_words = len(tokens)
    unique_tokens = len(set(tokens))
    avg_words_per_sentence = num_words / num_sent if num_sent else 0
    
    return {
        "n_sentences": num_sent,
        "words": num_words,
        "unique_tokens": unique_tokens,
        "avg_words_per_sentence": round(avg_words_per_sentence, 2)
    }

# Process CSV with spaCy row-by-row (much faster than concatenating)
def process_csv_stats_spacy_optimized(df_subset):
    """
    Process CSV data with spaCy row-by-row to avoid memory issues.
    Each sentence is already split in the CSV, so we process individually.
    """
    total_pairs = len(df_subset)
    corrected_pairs = df_subset['corrected'].sum()
    left_as_is = total_pairs - corrected_pairs
    
    # Count sentences (each row = 1 sentence pair = 2 sentences)
    total_sentences = total_pairs * 2
    corrected_sentences = corrected_pairs * 2
    uncorrected_sentences = left_as_is * 2
    
    all_tokens = []
    
    # Process src column row-by-row
    for idx, text in enumerate(df_subset['src'].fillna('')):
        if text and str(text).strip():
            try:
                doc = nlp(str(text))
                tokens = [tok.text for tok in doc if tok.is_alpha]
                all_tokens.extend(tokens)
            except Exception as e:
                # Skip problematic rows
                continue
    
    # Process tgt column row-by-row
    for idx, text in enumerate(df_subset['tgt'].fillna('')):
        if text and str(text).strip():
            try:
                doc = nlp(str(text))
                tokens = [tok.text for tok in doc if tok.is_alpha]
                all_tokens.extend(tokens)
            except Exception as e:
                # Skip problematic rows
                continue
  
    print()  # New line after progress indicators
    
    num_words = len(all_tokens)
    unique_tokens = len(set(all_tokens))
    avg_words_per_sentence = num_words / total_sentences if total_sentences else 0
    
    return {
        "n_sentence_pairs": total_pairs,
        "n_sentences": total_sentences,
        "words": num_words,
        "unique_tokens": unique_tokens,
        "avg_words_per_sentence": round(avg_words_per_sentence, 2),
        "corrected_pairs": int(corrected_pairs),
        "left_as_is": int(left_as_is),
        "corrected_pairs_pct": f"{round(corrected_pairs / total_pairs * 100, 2)}%" if total_pairs else "0%",
        "corrected_sentences": int(corrected_sentences),
        "uncorrected_sentences": int(uncorrected_sentences),
        "corrected_sentences_pct": round(corrected_sentences / total_sentences * 100, 2) if total_sentences else 0
    }

def compute_corpus_stats(source="both", csv_path="all_corpora.csv"):
    """
    Compute statistics on corpus data.
    
    Args:
        source: "txt", "csv", or "both"
        csv_path: Path to CSV file
    
    Returns:
        DataFrame with statistics
    """
    corpus_files = {
        "LEONIDE": "LEONIDE_full.txt",
        "Kolipsi_1_L1": "Kolipsi_1_L1_full.txt",
        "Kolipsi_1_L2": "Kolipsi_1_L2_full.txt",
        "Kolipsi_2": "Kolipsi_2_full.txt"
    }
    
    results = []
    
    # Process TXT files
    if source in ["txt", "both"]:
        print("=== Processing TXT files ===\n")
        
        # Individual corpora
        txt_results = []
        for name, path in corpus_files.items():
            try:
                stats = process_corpus_spacy(path)
                stats["corpus"] = name
                stats["source"] = "TXT"
                txt_results.append(stats)
                results.append(stats)
                print(f"✓ Processed {name}")
            except FileNotFoundError:
                print(f"✗ File not found: {path}")
            except ValueError as e:
                if "exceeds maximum" in str(e):
                    print(f"✗ {name}: Text too large, skipping detailed analysis")
                else:
                    raise
        
        # Whole TXT corpus - aggregate from individual stats
        if txt_results:
            all_stats = {
                "corpus": "WHOLE_CORPUS",
                "source": "TXT",
                "n_sentences": sum(r["n_sentences"] for r in txt_results),
                "words": sum(r["words"] for r in txt_results),
                "unique_tokens": sum(r["unique_tokens"] for r in txt_results),
                "avg_words_per_sentence": 0
            }
            if all_stats["n_sentences"] > 0:
                all_stats["avg_words_per_sentence"] = round(
                    all_stats["words"] / all_stats["n_sentences"], 2
                )
            results.append(all_stats)
            print(f"✓ Aggregated combined TXT corpus stats")
    
    # Process CSV file
    if source in ["csv", "both"]:
        print("\n=== Processing CSV file ===\n")
        
        try:
            df_csv = pd.read_csv(csv_path, encoding="utf-8")
            
            # Individual corpora from CSV
            corpus_names = sorted(df_csv['corpus'].unique())
            
            for corpus_name in corpus_names:
                df_subset = df_csv[df_csv['corpus'] == corpus_name]
                stats = process_csv_stats_spacy_optimized(df_subset)
                stats["corpus"] = corpus_name
                stats["source"] = "CSV"
                results.append(stats)
            
            # Whole CSV corpus
            all_csv_stats = process_csv_stats_spacy_optimized(df_csv)
            all_csv_stats["corpus"] = "WHOLE_CORPUS"
            all_csv_stats["source"] = "CSV"
            results.append(all_csv_stats)

        except FileNotFoundError:
            print(f"✗ CSV file not found: {csv_path}")
    
    # Convert to DataFrame
    df_results = pd.DataFrame(results)
    
    # Reorder columns for better readability
    base_cols = ["corpus"]
    
    if "n_sentence_pairs" in df_results.columns:
        other_cols = ["n_sentence_pairs", "n_sentences", "words", "unique_tokens", 
                     "avg_words_per_sentence", "corrected_pairs", "left_as_is", 
                     "corrected_pairs_pct"]
    else:
        other_cols = ["n_sentences", "words", "unique_tokens", "avg_words_per_sentence"]
    
    available_cols = base_cols + [col for col in other_cols if col in df_results.columns]
    df_results = df_results[available_cols]
    
    return df_results

def compute_corrected_only_stats(csv_path="all_corpora.csv"):
    """
    Compute statistics for corrected pairs only.
    
    Returns:
        DataFrame with corrected-only statistics
    """
    try:
        df_csv_full = pd.read_csv(csv_path, encoding="utf-8")
        df_corrected_only = df_csv_full[df_csv_full['corrected'] == True]
        
        if len(df_corrected_only) == 0:
            print("No corrected pairs found in the dataset.")
            return pd.DataFrame()
        
        corrected_stats = []
        corpus_names = sorted(df_corrected_only['corpus'].unique())
        
        # Per-subcorpus stats for corrected pairs only
        for corpus_name in corpus_names:
            df_subset = df_corrected_only[df_corrected_only['corpus'] == corpus_name]
            
            all_tokens = []
            
            # Process src
            for text in df_subset['src'].fillna(''):
                if text and str(text).strip():
                    try:
                        doc = nlp(str(text))
                        tokens = [tok.text for tok in doc if tok.is_alpha]
                        all_tokens.extend(tokens)
                    except:
                        continue
            
            # Process tgt
            for text in df_subset['tgt'].fillna(''):
                if text and str(text).strip():
                    try:
                        doc = nlp(str(text))
                        tokens = [tok.text for tok in doc if tok.is_alpha]
                        all_tokens.extend(tokens)
                    except:
                        continue
            
            num_words = len(all_tokens)
            unique_tokens = len(set(all_tokens))
            total_sentences = len(df_subset) * 2  # Each pair = 2 sentences
            avg_words = num_words / total_sentences if total_sentences else 0
            
            corrected_stats.append({
                'corpus': corpus_name,
                'corrected_pairs': len(df_subset),
                'words': num_words,
                'unique_tokens': unique_tokens,
                'avg_words_per_sentence': round(avg_words, 2)
            })
            print(f"  ✓ Done\n")
        
        # Whole corpus corrected pairs
        print(f"  Processing ALL corrected pairs ({len(df_corrected_only):,} rows)...")
        all_tokens_corrected = []
        
        for text in df_corrected_only['src'].fillna(''):
            if text and str(text).strip():
                try:
                    doc = nlp(str(text))
                    tokens = [tok.text for tok in doc if tok.is_alpha]
                    all_tokens_corrected.extend(tokens)
                except:
                    continue
        
        for text in df_corrected_only['tgt'].fillna(''):
            if text and str(text).strip():
                try:
                    doc = nlp(str(text))
                    tokens = [tok.text for tok in doc if tok.is_alpha]
                    all_tokens_corrected.extend(tokens)
                except:
                    continue
        
        num_words_all = len(all_tokens_corrected)
        unique_tokens_all = len(set(all_tokens_corrected))
        total_sentences_all = len(df_corrected_only) * 2
        avg_words_all = num_words_all / total_sentences_all if total_sentences_all else 0
        
        corrected_stats.append({
            'corpus': 'ALL_CORRECTED',
            'corrected_pairs': len(df_corrected_only),
            'words': num_words_all,
            'unique_tokens': unique_tokens_all,
            'avg_words_per_sentence': round(avg_words_all, 2)
        })
        
        return pd.DataFrame(corrected_stats)
        
    except FileNotFoundError:
        print(f"✗ CSV file not found: {csv_path}")
        return pd.DataFrame()


# CONTROL PANEL - CUSTOMIZE YOUR OUTPUT HERE

# 1. Choose data source
SOURCE = "csv"  # Options: "txt", "csv", or "both"

# 2. Choose which DataFrames to display (True = show, False = hide)
SHOW_MAIN_STATS = True              # Main corpus statistics table
SHOW_CORRECTION_BREAKDOWN = True    # Correction stats by subcorpus
SHOW_CORRECTION_SUMMARY = True      # Overall correction summary
SHOW_CORRECTED_ONLY_STATS = True    # Detailed stats for corrected pairs only


# MAIN EXECUTION - THIS ACTUALLY RUNS THE CODE
if __name__ == "__main__":
    print("\n" + "="*80)
    print(f"CORPUS STATISTICS (Source: {SOURCE.upper()})")
    print("="*80)

    # 1. Main Statistics
    if SHOW_MAIN_STATS:
        df_stats = compute_corpus_stats(source=SOURCE, csv_path="all_corpora.csv")
        display(df_stats)

    # Additional CSV-specific analyses
    if SOURCE in ["csv", "both"]:
        
        # 2. Correction Breakdown by Subcorpus
        if SHOW_CORRECTION_BREAKDOWN:
            print("\n" + "="*80)
            print("CORRECTION STATISTICS BREAKDOWN")
            print("="*80)
            
            try:
                df_csv_full = pd.read_csv("all_corpora.csv", encoding="utf-8")
                
                print("\n--- By Subcorpus ---")
                correction_by_corpus = df_csv_full.groupby('corpus')['corrected'].agg([
                    ('total_pairs', 'count'),
                    ('corrected_pairs', 'sum'),
                    ('left_as_is', lambda x: (~x).sum()),
                    ('corrected_pct', lambda x: f"{round(x.sum() / len(x) * 100, 2)}%")
                ]).reset_index()
                
                display(correction_by_corpus)
                
            except FileNotFoundError:
                print("✗ CSV file not found for correction analysis")
        
        # 3. Overall Correction Summary
        if SHOW_CORRECTION_SUMMARY:
            try:
                if 'df_csv_full' not in locals():
                    df_csv_full = pd.read_csv("all_corpora.csv", encoding="utf-8")
                
                print("\n--- Whole Corpus ---")
                total_pairs = len(df_csv_full)
                corrected_pairs = df_csv_full['corrected'].sum()
                left_as_is = total_pairs - corrected_pairs
                
                overall_stats = pd.DataFrame([{
                    'Metric': 'Total Sentence Pairs',
                    'Count': total_pairs,
                    'Percentage': '100.00%'
                }, {
                    'Metric': 'Corrected Pairs (True)',
                    'Count': int(corrected_pairs),
                    'Percentage': f"{corrected_pairs/total_pairs*100:.2f}%"
                }, {
                    'Metric': 'Left-As-Is Pairs (False)',
                    'Count': int(left_as_is),
                    'Percentage': f"{left_as_is/total_pairs*100:.2f}%"
                }])
                
                display(overall_stats)
                
            except FileNotFoundError:
                print("✗ CSV file not found for correction analysis")
        
        # 4. Corrected Pairs Only - Detailed Stats
        if SHOW_CORRECTED_ONLY_STATS:
            print("\n" + "="*80)
            print("CORRECTED PAIRS ONLY - DETAILED STATISTICS")
            print("="*80)
            
            df_corrected_stats = compute_corrected_only_stats(csv_path="all_corpora.csv")
            if not df_corrected_stats.empty:
                display(df_corrected_stats)


CORPUS STATISTICS (Source: CSV)

=== Processing CSV file ===








,corpus,n_sentence_pairs,n_sentences,words,unique_tokens,avg_words_per_sentence,corrected_pairs,left_as_is,corrected_pairs_pct
0,Kolipsi_1_L1,5017,10034,137209,6757,13.67,987,4030,19.67%
1,Kolipsi_1_L2,5081,10162,112658,5640,11.09,1835,3246,36.11%
2,Kolipsi_2,13392,26784,330530,7005,12.34,4816,8576,35.96%
3,LEONIDE,4399,8798,118080,7369,13.42,2022,2377,45.96%
4,WHOLE_CORPUS,27889,55778,698477,18168,12.52,9660,18229,34.64%



CORRECTION STATISTICS BREAKDOWN

--- By Subcorpus ---


,corpus,total_pairs,corrected_pairs,left_as_is,corrected_pct
0,Kolipsi_1_L1,5017,987,4030,19.67%
1,Kolipsi_1_L2,5081,1835,3246,36.11%
2,Kolipsi_2,13392,4816,8576,35.96%
3,LEONIDE,4399,2022,2377,45.96%



--- Whole Corpus ---


,Metric,Count,Percentage
0,Total Sentence Pairs,27889,100.00%
1,Corrected Pairs (True),9660,34.64%
2,Left-As-Is Pairs (False),18229,65.36%



CORRECTED PAIRS ONLY - DETAILED STATISTICS
  ✓ Done

  ✓ Done

  ✓ Done

  ✓ Done

  Processing ALL corrected pairs (9,660 rows)...


,corpus,corrected_pairs,words,unique_tokens,avg_words_per_sentence
0,Kolipsi_1_L1,987,32873,3457,16.65
1,Kolipsi_1_L2,1835,49424,4153,13.47
2,Kolipsi_2,4816,142594,5084,14.80
3,LEONIDE,2022,62952,5846,15.57
4,ALL_CORRECTED,9660,287843,13116,14.90


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Merged alignment preparation script with stratified K-fold cross-validation using sklearn
Reads CSV, applies stratified sampling, and creates alignment input files
"""

import re
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

# ============================================================
# TOKENIZATION
# ============================================================

def simple_tokenize(text):
    """Fast regex tokenizer"""
    tokens = re.findall(r"\w+|[^\w\s]", text, re.UNICODE)
    return tokens

def segment_with_underscores(text):
    """Segment text with underscores between tokens"""
    return "_".join(simple_tokenize(text))

# ============================================================
# STRATIFIED SAMPLING WITH SKLEARN
# ============================================================

def stratified_split(df, k_folds=5, random_state=42):
    """
    Perform stratified K-fold split using sklearn.model_selection.StratifiedKFold
    Maintains:
    - 65% left-as-is (corrected=False) / 35% corrected (corrected=True)
    - Same corpus proportions in each fold
    
    Parameters:
    -----------
    df : DataFrame with columns 'corpus', 'corrected', 'src', 'tgt'
    k_folds : number of folds for cross-validation
    random_state : random seed for reproducibility
    
    Returns:
    --------
    list of tuples: [(train_df, test_df), ...] for each fold
    """
    # Target proportions for reference
    target_corpus_props = {
        'Kolipsi_1_L1': 0.2287,
        'Kolipsi_1_L2': 0.2728,
        'Kolipsi_2': 0.3076,
        'LEONIDE': 0.1908
    }
    
    target_correction_props = {
        False: 0.65,  # left-as-is
        True: 0.35    # corrected
    }
    
    # Create stratification key combining corpus and correction status
    df['strat_key'] = df['corpus'].astype(str) + '_' + df['corrected'].astype(str)
    
    # Initialize StratifiedKFold from sklearn
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_state)
    
    # Get indices for X (features) and y (stratification labels)
    X = df.index.to_numpy()
    y = df['strat_key'].to_numpy()
    
    splits = []
    
    # Generate train/test splits using sklearn
    for fold_idx, (train_indices, test_indices) in enumerate(skf.split(X, y), 1):
        train_df = df.iloc[train_indices].copy()
        test_df = df.iloc[test_indices].copy()
        
        splits.append((train_df, test_df))
        
        # Print statistics
        print(f"\n=== FOLD {fold_idx}/{k_folds} ===")
        print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")
        
        for split_name, split_df in [("Train", train_df), ("Test", test_df)]:
            print(f"\n{split_name} set:")
            print("  Corpus distribution:")
            for corpus in target_corpus_props.keys():
                count = (split_df['corpus'] == corpus).sum()
                prop = count / len(split_df) if len(split_df) > 0 else 0
                print(f"    {corpus}: {count} ({prop:.2%}) [target: {target_corpus_props[corpus]:.2%}]")
            
            print("  Correction distribution:")
            for corrected_val in [False, True]:
                count = (split_df['corrected'] == corrected_val).sum()
                prop = count / len(split_df) if len(split_df) > 0 else 0
                label = "left-as-is" if not corrected_val else "corrected"
                print(f"    {label}: {count} ({prop:.2%}) [target: {target_correction_props[corrected_val]:.2%}]")
    
    # Drop temporary column
    df.drop('strat_key', axis=1, inplace=True)
    
    return splits

# ============================================================
# SAVE ALIGNMENT FILES
# ============================================================

def save_alignment_files(df, out_dir, set_name):
    """
    Save alignment input files from DataFrame
    
    Parameters:
    -----------
    df : DataFrame with 'src' and 'tgt' columns
    out_dir : output directory
    set_name : name for the file set (e.g., 'train', 'test')
    """
    os.makedirs(out_dir, exist_ok=True)
    
    print(f"\nCreating alignment files for: {set_name}")
    print(f"Total sentence pairs: {len(df)}")
    
    # File paths
    src_path = os.path.join(out_dir, f"{set_name}_src.txt")
    tgt_path = os.path.join(out_dir, f"{set_name}_tgt.txt")
    src_tok = os.path.join(out_dir, f"{set_name}_src.tok")
    tgt_tok = os.path.join(out_dir, f"{set_name}_tgt.tok")
    src_segm = os.path.join(out_dir, f"{set_name}_src.segm")
    tgt_segm = os.path.join(out_dir, f"{set_name}_tgt.segm")
    
    with open(src_path, "w", encoding="utf-8") as fsrc, \
         open(tgt_path, "w", encoding="utf-8") as ftgt, \
         open(src_tok, "w", encoding="utf-8") as fsrc_tok, \
         open(tgt_tok, "w", encoding="utf-8") as ftgt_tok, \
         open(src_segm, "w", encoding="utf-8") as fsrc_seg, \
         open(tgt_segm, "w", encoding="utf-8") as ftgt_seg:
        
        for _, row in df.iterrows():
            s = str(row['src']).strip()
            t = str(row['tgt']).strip()
            
            # Write plain text
            fsrc.write(s + "\n")
            ftgt.write(t + "\n")
            
            # Tokenized
            s_tok = simple_tokenize(s)
            t_tok = simple_tokenize(t)
            fsrc_tok.write(" ".join(s_tok) + "\n")
            ftgt_tok.write(" ".join(t_tok) + "\n")
            
            # Segmented (underscore-separated)
            fsrc_seg.write(segment_with_underscores(s) + "\n")
            ftgt_seg.write(segment_with_underscores(t) + "\n")
    
    print(f"✓ Alignment files created in: {out_dir}")
    print(f"  - {set_name}_src.txt / {set_name}_tgt.txt (plain)")
    print(f"  - {set_name}_src.tok / {set_name}_tgt.tok (tokenized)")
    print(f"  - {set_name}_src.segm / {set_name}_tgt.segm (segmented)")

# ============================================================
# MAIN PROCESSING FUNCTION
# ============================================================

def process_csv_to_alignment(csv_path, out_dir="alignment_output", k_folds=5, random_state=42):
    """
    Main function: Load CSV, apply stratified sampling with sklearn, create alignment files
    
    Parameters:
    -----------
    csv_path : path to input CSV file
    out_dir : output directory for alignment files
    k_folds : number of cross-validation folds
    random_state : random seed
    """
    # Load CSV
    print(f"Loading CSV from: {csv_path}")
    df = pd.read_csv(csv_path)
    
    # Verify required columns
    required_cols = ['src', 'tgt', 'corpus', 'corrected']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    print(f"\nTotal sentences loaded: {len(df)}")
    print(f"Columns: {list(df.columns)}")
    
    # Convert 'corrected' to boolean if needed
    if df['corrected'].dtype == 'object':
        df['corrected'] = df['corrected'].map({'True': True, 'False': False, True: True, False: False})
    
    # Perform stratified split with sklearn
    print("\n" + "="*60)
    print("PERFORMING STRATIFIED K-FOLD SPLIT (using sklearn)")
    print("="*60)
    splits = stratified_split(df, k_folds=k_folds, random_state=random_state)
    
    # Save alignment files for each fold
    print("\n" + "="*60)
    print("CREATING ALIGNMENT FILES")
    print("="*60)
    
    for fold_idx, (train_df, test_df) in enumerate(splits, 1):
        fold_dir = os.path.join(out_dir, f"fold_{fold_idx}")
        
        save_alignment_files(train_df, fold_dir, f"train")
        save_alignment_files(test_df, fold_dir, f"test")
    
    print("\n" + "="*60)
    print("✓ ALL PROCESSING COMPLETE")
    print("="*60)
    print(f"Output directory: {out_dir}")
    print(f"Number of folds: {k_folds}")
    print(f"Method: sklearn.model_selection.StratifiedKFold")
    
    return splits

# ============================================================
# EXAMPLE USAGE
# ============================================================

if __name__ == "__main__":
    # Example usage
    csv_file = "your_data.csv"  # Replace with your CSV path
    output_directory = "alignment_output"
    
    # Process CSV and create alignment files
    splits = process_csv_to_alignment(
        csv_path=csv_file,
        out_dir=output_directory,
        k_folds=5,
        random_state=42
    )
    
    print("\nYou can access the splits programmatically:")
    print("  splits[0] = (train_df_fold1, test_df_fold1)")
    print("  splits[1] = (train_df_fold2, test_df_fold2)")
    print("  etc.")